# Importing functions

In [30]:
import pandas as pd 
import numpy as np
import pandasql as ps
import os
import re
import random 
import numpy as np
import warnings
import tensorflow as tf 
from imblearn.over_sampling import RandomOverSampler
from sklearn.compose import make_column_transformer
from tensorflow.keras import layers
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from preprocess_days_stats import preprocess_match_days
from preprocess_time_serie import preprocess_teams, create_time_series_features
from preprocess_time_series_features import preprocess_features_time_series, create_fast_preprocessing_ts, preprocess_features_time_series_odds, create_fast_preprocessing_ts_odds
from helper_functions_tensorflow import CSVLoggerCallback, CSVLoggerCallbackParams

# Ignora tutti i warning temporaneamente
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)

# Adding the new results to the dataframe

In [32]:
# Leggi il file CSV
df = pd.read_csv(r"C:\Users\Hp\Documents\Serie_A_dump\csv_predictions\december_four.csv", parse_dates=['Date'], dayfirst=True)

# Crea un DataFrame con 10 righe di zeri
new_rows = pd.DataFrame(0, index=range(10), columns=df.columns)

today_date = '2023-12-10'

# Assegna valori specifici alle colonne 'Div', 'Date', 'Time', 'HomeTeam', 'AwayTeam' per ciascuna riga
new_rows.loc[0, ['Div', 'Date', 'Time', 'HomeTeam', 'AwayTeam']] = ['I1', f'{today_date}', '17:30', 'Juventus', 'Napoli']
new_rows.loc[1, ['Div', 'Date', 'Time', 'HomeTeam', 'AwayTeam']] = ['I1', f'{today_date}', '17:30', 'Verona', 'Lazio']
new_rows.loc[2, ['Div', 'Date', 'Time', 'HomeTeam', 'AwayTeam']] = ['I1', f'{today_date}', '17:30', 'Atalanta', 'Milan']
new_rows.loc[3, ['Div', 'Date', 'Time', 'HomeTeam', 'AwayTeam']] = ['I1', f'{today_date}', '17:30', 'Inter', 'Udinese']
new_rows.loc[4, ['Div', 'Date', 'Time', 'HomeTeam', 'AwayTeam']] = ['I1', f'{today_date}', '17:30', 'Frosinone', 'Torino']
new_rows.loc[5, ['Div', 'Date', 'Time', 'HomeTeam', 'AwayTeam']] = ['I1', f'{today_date}', '17:30', 'Monza', 'Genoa']
new_rows.loc[6, ['Div', 'Date', 'Time', 'HomeTeam', 'AwayTeam']] = ['I1', f'{today_date}', '17:30', 'Salernitana', 'Bologna']
new_rows.loc[7, ['Div', 'Date', 'Time', 'HomeTeam', 'AwayTeam']] = ['I1', f'{today_date}', '17:30', 'Roma', 'Fiorentina']
new_rows.loc[8, ['Div', 'Date', 'Time', 'HomeTeam', 'AwayTeam']] = ['I1', f'{today_date}', '17:30', 'Empoli', 'Lecce']
new_rows.loc[9, ['Div', 'Date', 'Time', 'HomeTeam', 'AwayTeam']] = ['I1', f'{today_date}', '17:30', 'Cagliari', 'Sassuolo']


# Aggiungi le nuove righe al DataFrame esistente
new_csv = pd.concat([df, new_rows], ignore_index=True)
new_csv['Date'] = pd.to_datetime(new_csv['Date'], format='%Y-%m-%d')

# Salva il DataFrame aggiornato su un nuovo file CSV
new_csv.to_csv(r"C:\Users\Hp\Serie_A\csv_predictions\december_four.csv")

len(new_csv['HomeTeam'].unique()), len(new_csv['AwayTeam'].unique())


(20, 20)

In [37]:
#preprocess the data
df_giornate = preprocess_match_days(r"C:\Users\Hp\Serie_A\csv_predictions")
num_features = 'less'
num_giornate = 4
Statistiche_squadre_dict = preprocess_teams(dataframe = df_giornate)
df_Serie_A = create_time_series_features(num_features, Statistiche_squadre_dict, df_giornate, num_giornate).dropna()

(Train_teams_encoded, Valid_teams_encoded, Test_teams_encoded, Train_labels_encoded, Valid_labels_encoded, Test_labels_encoded, 
    Train_dict_features_norm, Valid_dict_features_norm, Test_dict_features_norm, Train_teams, Valid_teams, Test_teams, Train_labels, Valid_labels, 
    Test_labels, Train_dict_features, Valid_dict_features, Test_dict_features, Train_df, Valid_df, Test_df, 
    Train_odds_norm, Valid_odds_norm, Test_odds_norm) = preprocess_features_time_series_odds(df_Serie_A, num_features, random_state=False)

feature_input_shape = Test_dict_features_norm[list(Test_dict_features_norm.keys())[0]].shape[1]
Train_teams_shape = Test_teams_encoded.shape[1]

Dataset_train_norm, Dataset_valid_norm, Dataset_test_norm = create_fast_preprocessing_ts_odds(Train_teams_encoded, Train_dict_features_norm, Train_labels_encoded,
                                                                                         Valid_teams_encoded, Valid_dict_features_norm,
                                                                    Valid_labels_encoded,Test_teams_encoded, Test_dict_features_norm, Test_labels_encoded, 
                                                                    Train_odds_norm, Valid_odds_norm, Test_odds_norm)

Reading file: december_four.csv
Reading file: I1 (0).csv
Reading file: I1 (1).csv
Reading file: I1 (10).csv
Reading file: I1 (11).csv
Reading file: I1 (12).csv
Reading file: I1 (13).csv
Reading file: I1 (14).csv
Reading file: I1 (15).csv
Reading file: I1 (16).csv
Reading file: I1 (17).csv
Reading file: I1 (2).csv
Reading file: I1 (3).csv
Reading file: I1 (4).csv
Reading file: I1 (5).csv
Reading file: I1 (6).csv
Reading file: I1 (7).csv
Reading file: I1 (8).csv
Reading file: I1 (9).csv
preprocessing finished!
utilizzando meno features
preprocess finished
utilizzando meno features
utilizzando meno features
utilizzando meno features


In [38]:
#import the model 
odds_model = tf.keras.models.load_model(r'c:\Users\Hp\Serie_A\model_experiments\model_odds_time_series')


In [41]:
Valid_labels.shape

(6225, 1)

In [39]:
## Visualizziamo un po' di risultati 
model_odds_new_pred_probs = odds_model.predict((Dataset_test_norm))
model_odds_new_prob = model_odds_new_pred_probs.max(axis=1)
model_odds_new_predictions = model_odds_new_pred_probs.argmax(axis=1)
model_odds_new_compare = pd.DataFrame({
                                'stagione': list( Test_df['stagione'] ),
                                'hometeam': list( Test_df['hometeam'] ),
                                'awayteam': list( Test_df['awayteam'] ),
                                'preds': model_odds_new_predictions, 
                                'result': Test_labels, 
                                'best_pred_prob': model_odds_new_prob,
                                'home_win_odds': list( Test_df['home_win_odds'] ),
                                'draw_odds': list( Test_df['draw_odds'] ),
                                'away_win_odds': list( Test_df['away_win_odds'])
                                })

195/195 [==============================] - 1s 3ms/step


ValueError: 2